In [59]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest, chi2,f_classif,mutual_info_classif
import jieba
jieba.load_userdict('userword.txt')
import numpy as np

In [60]:
import re

## 清理文本中的各种标点：
def get_clean_text(text):
    text = text.strip()
    text = re.sub(r'^(1?[0-9]\.?、?\)?）?)?\*?-?·?•?(\\t)?', '', text)
    text = re.sub(r'[;；。]$', '', text)
    text = re.sub(r'^\s*', '', text)
    text = text.replace(' ','')
    return text

## 把文本转化成句子列表：
def text2sentences(text):
    raw_data = []
    for line in text.readlines():
        raw_data.append(get_clean_text(line.decode('utf-8')))
    raw_data = list(set(raw_data))
    return raw_data

###########seg_sentence function#####################################################
# 创建停用词list
def stopwordslist(filepath):
    stopwords = [line.strip() for line in open(filepath, 'r', encoding='utf-8').readlines()]
    return stopwords
# 对句子进行分词
def seg_sentence(sentence):
    sentence_seged = jieba.cut(sentence.strip())
    stopwords = stopwordslist('stopwords.txt')  # 这里加载停用词的路径
    outstr = ''
    for word in sentence_seged:
        if word not in stopwords:
            outstr += word+'/'
    outstr = outstr.split('/')
    return outstr

In [65]:
def feature_words_extraction(word_cuted, labels,score_func, func=TfidfVectorizer, **kwargs):
    """
    传入文本和y值，用来构建模型和提取参数
    :param word_cuted: dataframe 一列，每个值为分好词的文本，词汇以空格隔开
    :param labels: numpy array of shape [n_samples]
    :param keyword_ratio: 关键词提取比例
    :param func: Vectorizer 的 方法，默认为tf-idf
    :param kwargs: TfidfVectorizer 的 方法参数
    """
    ############# 先通过tf-idf来获取文本代表词
    vectorizer = func(**kwargs)
    # 获取文本矩阵（词频，tf-idf等）
    vec_matrix = vectorizer.fit_transform(word_cuted)
    # 获取特征词
    feature_names = vectorizer.get_feature_names()
#     print("Orig_feature_names:\n",feature_names)
#     print("-------------------")
    # 计算特征词数
    # keyword_num = int(keyword_ratio * len(feature_names))
#     keyword_num = int(keyword_ratio * len(labels))
    keyword_num = 150
    print('{}个特征词，{}个样本'.format(keyword_num, len(labels)))
    if keyword_num == 0:
        raise TrainException("特征词不足，请增加传入的数据量")
    # 根据具体方法，获取selector对象
    if score_func == 'chi2':
        selector = SelectKBest(chi2, k=keyword_num)
    elif score_func == 'f_classif':
        selector = SelectKBest(f_classif, k=keyword_num)
    elif score_func == 'mutual_info_classif':
        selector = SelectKBest(mutual_info_classif, k=keyword_num)
    # 获取卡方矩阵
    ch2_matrix = selector.fit_transform(vec_matrix, labels)
    # 抽取特征词
    feature_names = [feature_names[i] for i in selector.get_support(indices=True)]
    print("用%s查到特征词：\n"%score_func,feature_names)
    print("-------------------")

In [66]:
f_yes = 'C:/Users/x1c/Desktop/猎上顺丰简历项目/正负样本/产品经理合格.txt'
f_no = 'C:/Users/x1c/Desktop/猎上顺丰简历项目/正负样本/产品经理不合格.txt'
##### 把文本转化成干净的词列表：#############
def from_text_to_clean_words(filepath):
    f = open(filepath,'r',encoding='utf-8')
    text_words = seg_sentence(get_clean_text(f.read()))
    return text_words
t_y = from_text_to_clean_words(f_yes)
t_n = from_text_to_clean_words(f_no)

In [67]:
####### 生成对应的标签列表，并将正负样本数据合并：##########
yes_labels = list(np.ones(len(t_y)))
no_labels = list(np.zeros(len(t_n)))
text_cuted = t_y+t_n
labels = yes_labels+no_labels
print(len(text_cuted))
print(len(labels))

9829
9829


In [68]:
score_func_list = ['chi2','f_classif','mutual_info_classif']
for func in score_func_list:
    feature_words_extraction(text_cuted, labels, score_func=func)

150个特征词，9829个样本
用chi2查到特征词：
 ['04', '2017', '________________________________________', 'cn', 'erp', 'h5', 'http', 'https', 'it', 'k12', 'pc端', 'web', 'www', '一期', '一款', '七期', '下属', '专家', '业务', '业绩', '中移', '书写', '买金', '二期', '交互', '交互设计', '产品', '产品功能', '产品经理', '产品部', '人员', '人数', '信息化', '充值', '全网', '公司', '分析报告', '制定', '前台', '前端开发', '功能', '助教', '北京', '升级', '华润集团', '原因', '原型', '反馈', '后台', '国际', '地产', '地区', '场景', '基金', '增加', '大师', '学生', '官网', '实时', '客服', '宣传', '对接', '对象', '市场调研', '市场部', '建置', '德稻', '快速', '战略', '技能', '报告', '报表', '接入', '推荐', '提升', '搭建', '撰写', '操作', '支付', '改版', '效率', '教学', '数据', '数据分析', '方案', '方案设计', '时间', '明源', '期望', '期间', '架构', '梳理', '模块', '模式', '活动', '涵盖', '独立', '环节', '现有', '理财师', '用户', '电子商务', '监控', '直聘', '研发', '确保', '确认', '禅道', '程序', '立项', '简介', '简历', '管控', '精通', '系统', '线上', '经销商', '经验', '结算', '统一', '职务', '职责', '袋鼠', '视觉', '视频', '订单', '设计', '设计产品', '评估', '课程', '调研', '财务', '账号', '费用', '资金', '跨境电商', '跨部门', '输出', '运维', '部门', '金蝶', '金融', '集团', '需求', '页面', '项目', '风控', '高保真', '